In [1]:
# all imports must go here
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import os
import shutil
import glob
import random
import cv2 as cv
import sys
from tqdm import tqdm
import json
import re

In [2]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/FeedRecSys

/content/drive/My Drive/FeedRecSys


#Load the text from the JSON file

In [4]:


# Opening JSON file
f = open('tops_fashion.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  



# Pretty Printing JSON string back
# print(json.dumps(data, indent = 4, sort_keys=True))

In [5]:
import pandas as pd
  
# reading the file
df = pd.read_json("tops_fashion.json")
  
# displaying the DataFrame
df.head()

,sku,asin,product_type_name,formatted_price,author,color,brand,publisher,availability,reviews,large_image_url,availability_type,small_image_url,editorial_review,title,model,medium_image_url,manufacturer,editorial_reivew
0,None,B016I2TS4W,SHIRT,None,None,None,FNC7C,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Minions Como Superheroes Ironman Women's O Nec...,Minions Como Superheroes Ironman Long Sleeve R...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
1,None,B01N49AI08,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Izo Tunic,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
2,None,B01JDPCOHO,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Won Top,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
3,None,B01N19U5H5,SHIRT,None,None,None,Focal18,None,None,"[True, https://www.amazon.com/reviews/iframe?a...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,100% Brand New & Fashion<br> Quantity: 1 Piece...,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
4,None,B004GSI2OS,SHIRT,$26.26,None,Onyx Black/ Stone,FeatherLite,None,Usually ships in 6-10 business days,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,now,https://images-na.ssl-images-amazon.com/images...,,Featherlite Ladies' Long Sleeve Stain Resistan...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN


In [6]:
null_counts = df.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

author               183137
sku                  182775
editorial_review     180380
availability         158606
availability_type    158579
formatted_price      154743
publisher            140239
manufacturer         140239
model                120768
color                118182
editorial_reivew       2758
brand                   151
dtype: int64

In [7]:
df.columns

Index(['sku', 'asin', 'product_type_name', 'formatted_price', 'author',
       'color', 'brand', 'publisher', 'availability', 'reviews',
       'large_image_url', 'availability_type', 'small_image_url',
       'editorial_review', 'title', 'model', 'medium_image_url',
       'manufacturer', 'editorial_reivew'],
      dtype='object')

In [8]:
df1 = df.drop(['author', 'formatted_price','reviews','sku', 'publisher','availability','availability_type', 'small_image_url', 'model', 'medium_image_url', 'manufacturer','editorial_reivew', 'large_image_url'], axis=1)

In [9]:
df1.drop_duplicates(subset=['brand'])

,asin,product_type_name,color,brand,editorial_review,title
0,B016I2TS4W,SHIRT,None,FNC7C,Minions Como Superheroes Ironman Women's O Nec...,Minions Como Superheroes Ironman Long Sleeve R...
1,B01N49AI08,SHIRT,None,FIG Clothing,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Izo Tunic
3,B01N19U5H5,SHIRT,None,Focal18,100% Brand New & Fashion<br> Quantity: 1 Piece...,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...
4,B004GSI2OS,SHIRT,Onyx Black/ Stone,FeatherLite,,Featherlite Ladies' Long Sleeve Stain Resistan...
5,B00TAEHGGS,SHIRT,Grape,Fitscloth,Solid Modern Long Sleeve Tee. Quality Cotton T...,[Fits Cloth] Grape Solid Modern Long Sleeve Pl...
...,...,...,...,...,...,...
181533,B01LORYNNC,SHIRT,Navy,Fashion eNation,NaN,Pinstripe Oversize Button Down Blouse High Low...
181864,B01N2R8QBX,SHIRT,None,GUESS Factory,NaN,GUESS Factory Women's Naya Button-Down Shirt
182371,B06XYLNQ38,SHIRT,Guava Pink,Z,NaN,"Women's Cap Sleeve Ruffle Neck Top, Guava Pink..."
182629,B01L8MELXA,SHIRT,Metal shining,Rain's Pan Jacket,NaN,Rain's Pan®Women's Metalic Glossy Shining Long...


In [10]:
df1['title'] = df['title'].str.lower()

df1['editorial_review'] = df['editorial_review'].str.lower()

df1['brand'] = df['brand'].str.lower()

df1['color'] = df['color'].str.lower()

df1['product_type_name'] = df['product_type_name'].str.lower()


In [11]:
df1.head()

,asin,product_type_name,color,brand,editorial_review,title
0,B016I2TS4W,shirt,None,fnc7c,minions como superheroes ironman women's o nec...,minions como superheroes ironman long sleeve r...
1,B01N49AI08,shirt,None,fig clothing,sizing runs on the small side. fig® recommends...,fig clothing womens izo tunic
2,B01JDPCOHO,shirt,None,fig clothing,sizing runs on the small side. fig® recommends...,fig clothing womens won top
3,B01N19U5H5,shirt,None,focal18,100% brand new & fashion<br> quantity: 1 piece...,focal18 sailor collar bubble sleeve blouse shi...
4,B004GSI2OS,shirt,onyx black/ stone,featherlite,,featherlite ladies' long sleeve stain resistan...


In [12]:
values = {"editorial_review": "good choice"}
df1 = df1.fillna(value=values)

In [13]:
cl = df1['color'].tolist()
cl_new = []
for j in cl:
  if j==None:
    j = 'multicolor'
  else:
    j = j
  cl_new.append(j)



In [14]:
bl = df1['brand'].tolist()
bl_new = []
for k in bl:
  if k == None:
    k = 'amazon brand'
  else:
    k = k
  bl_new.append(k)

In [15]:
df1['color'] = cl_new
df1['brand'] = bl_new



In [16]:
df1.head()

,asin,product_type_name,color,brand,editorial_review,title
0,B016I2TS4W,shirt,multicolor,fnc7c,minions como superheroes ironman women's o nec...,minions como superheroes ironman long sleeve r...
1,B01N49AI08,shirt,multicolor,fig clothing,sizing runs on the small side. fig® recommends...,fig clothing womens izo tunic
2,B01JDPCOHO,shirt,multicolor,fig clothing,sizing runs on the small side. fig® recommends...,fig clothing womens won top
3,B01N19U5H5,shirt,multicolor,focal18,100% brand new & fashion<br> quantity: 1 piece...,focal18 sailor collar bubble sleeve blouse shi...
4,B004GSI2OS,shirt,onyx black/ stone,featherlite,,featherlite ladies' long sleeve stain resistan...


In [17]:
df1['product_type_name'].isnull().values.any(), df1['color'].isnull().values.any(), df1['brand'].isnull().values.any(), df1['title'].isnull().values.any(), df1['editorial_review'].isnull().values.any()

(False, False, False, False, False)

In [18]:
df1.editorial_review.replace('',"good choice",regex = True)

0         minions como superheroes ironman women's o nec...
1         sizing runs on the small side. fig® recommends...
2         sizing runs on the small side. fig® recommends...
3         100% brand new & fashion<br> quantity: 1 piece...
4                                               good choice
                                ...                        
183133                                          good choice
183134                                          good choice
183135                                          good choice
183136                                          good choice
183137                                          good choice
Name: editorial_review, Length: 183138, dtype: object

In [19]:
df1['product_type_name'].isnull().values.any(), df1['color'].isnull().values.any(), df1['brand'].isnull().values.any(), df1['title'].isnull().values.any()

(False, False, False, False)

In [20]:
def remove_spl_charact(inputstr):
  text = re.sub(r"[^a-zA-Z0-9 ]", "", inputstr)
  return text

In [21]:
edrv = df1['editorial_review'].tolist()

rvw_list = []
for rvw in edrv:
  rvw_list.append(remove_spl_charact(rvw))


df1.drop(['editorial_review'], axis=1, inplace=True)
rvw_list

['minions como superheroes ironman womens o neck long sleeve tshirtit is made of 100 cottonwell printed images by environmental healthy inkscustomized t shirts will be the best and sincere gifts for your family and friends',
 'sizing runs on the small side fig recommends sizing up fig apparel size chart grab a map and explore the city from dusk til dawn you will conquer the day with ease in this modern tunic this is part of the figvoyage collection regular fit tunic has a modern cut that hugs the body while still leaving room for movement soft cottonpoly blend with stretch knit allows for allday comfort upf 50 fabrication protects your skin during outdoor activity by blocking harmful ultraviolet a and ultraviolet b rays uva and uvb wide bateau neck quartersleeves with solid band under arms smooth flatlock seams allow for chafefree motion decorative floral fauxpockets with buttonsnap at hips curved extended hemline 89 cotton 8 spandex 3 polyester machine wash tumble dry made in canada m

In [22]:
# len(concat_list)
df1['editorial_review'] = rvw_list

df1.head()

,asin,product_type_name,color,brand,title,editorial_review
0,B016I2TS4W,shirt,multicolor,fnc7c,minions como superheroes ironman long sleeve r...,minions como superheroes ironman womens o neck...
1,B01N49AI08,shirt,multicolor,fig clothing,fig clothing womens izo tunic,sizing runs on the small side fig recommends s...
2,B01JDPCOHO,shirt,multicolor,fig clothing,fig clothing womens won top,sizing runs on the small side fig recommends s...
3,B01N19U5H5,shirt,multicolor,focal18,focal18 sailor collar bubble sleeve blouse shi...,100 brand new fashionbr quantity 1 piecebr ma...
4,B004GSI2OS,shirt,onyx black/ stone,featherlite,featherlite ladies' long sleeve stain resistan...,


In [23]:
source_folder = './16k_images'
dir = source_folder

train_files = os.listdir(os.path.join('/content/drive/My Drive/FeedRecSys/', dir))
len(train_files)

16032

In [24]:
dfasin_list = df1['asin'].tolist()


In [25]:
# difference_1 = set(dfasin_list).difference(set(train_files))
difference_2 = set(train_files).difference(set(dfasin_list))


list_difference = list(difference_2)

In [26]:
idx_list = []
for val in tqdm(list_difference):

  # print (df.index[df['B'] == 19].tolist())
  idx = (df1.index[df1['asin'] == val.split('.')[0]].tolist())
  # print(idx[0])
  idx_list.extend(idx)

100%|██████████| 16032/16032 [10:05<00:00, 26.46it/s]


In [27]:
my_new_df = df1.iloc[idx_list]

In [28]:
my_new_df.head()

,asin,product_type_name,color,brand,title,editorial_review
139476,B01KFMD748,shirt,multi-color,jerry leigh apparel,"jerry leigh apparel juniors short sleeve ""live...",good choice
73834,B01HU4NYY2,books_1973_and_later,black,vansty,vansty jeremy lin brook-lin 100% cotton t-shir...,good choice
142433,B074S9Q7VM,shirt,multi,libian,libian women top plus size 3x chevron multi co...,good choice
149085,B00MPWB6LS,shirt,vintage dark black,next level apparel,next level apparel women's soft deep v-neck t-...,good choice
96431,B00IY71MOE,shirt,black,sf,sf tshirt with drop detail - black - s,good choice


In [29]:
my_new_df['editorial_review'].unique()

array(['good choice',
       'long sleeve suede top in plus sizefashion comes in four sizes and these tops with leather are loose fit very comfortable laser cut design end of the sleeves and front and back bottom paisley cut design front is v neck suede suede with lace up trim which completes this top very rare and only few pieces are left from best amazon plus size boutique',
       'supernatural chibis sam dean and castiel womens o neck short sleeve tshirtit is made of 100 cottonwell printed images by environmental healthy inkscustomized t shirts will be the best and sincere gifts for your family and friends',
       'our brand crop top shirt is of high rank the design and the picture are very unique and special the best point is the quality and the process technology that will not let you disappointed please visit our shop to see more goods',
       '',
       'light and comfortable tank top with feminine slightly longer cut made from a soft functional material with wool and tencel 

In [31]:
# ptl = my_new_df['product_type_name'].tolist()
# cl = my_new_df['color'].tolist()
# bl = my_new_df['brand'].tolist()
# tl = my_new_df['title'].tolist()
al = my_new_df['asin'].tolist()
rl = my_new_df['editorial_review'].tolist()

concat_map = {}
for u,v in zip(al,rl):
  # print(len(x))
  # concat_ele = u+" "+v
  text = remove_spl_charact(v)
  # print(text)
  concat_map.update({u:text})



In [32]:
!pip3 install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 4.7 MB 16.3 MB/s 
     |████████████████████████████████| 1.3 MB 52.0 MB/s 
     |████████████████████████████████| 120 kB 60.5 MB/s 
     |████████████████████████████████| 6.6 MB 45.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a4d26ed9929b7cb57dd8be5bbb0e2ccc974ca889e0ce9f87c7e10c2fd1c49bf9
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [33]:
from sentence_transformers import SentenceTransformer

In [34]:
concat_list = list(concat_map.values())

l_a = []
for i in concat_list:
  l_a.append(len(i))

mean_length = int(np.floor(np.mean(l_a)))
mean_length

14

In [36]:
# print(concat_list[:200])

['good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'long sleeve suede top in plus sizefashion comes in four sizes and these tops with leather are loose fit very comfortable laser cut design end of the sleeves and front and back bottom paisley cut design front is v neck suede suede with lace up trim which completes this top very rare and only few pieces are left from best amazon plus size boutique', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'good choice', 'go

In [37]:

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(concat_list, show_progress_bar=True)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/501 [00:00<?, ?it/s]

In [43]:
review_emb_map = {}
for (k,v), embeddings in zip((concat_map.items()),encoded_data):
  sentences = k + " " + v
  # print(sentences)
  review_emb_map.update({sentences:embeddings})

In [44]:
mykeys = list(review_emb_map.keys())
mykeys

['B01KFMD748 good choice',
 'B01HU4NYY2 good choice',
 'B074S9Q7VM good choice',
 'B00MPWB6LS good choice',
 'B00IY71MOE good choice',
 'B01MY98XAC good choice',
 'B0722YJ8VF good choice',
 'B01DAHCT3U good choice',
 'B073P9D2QP good choice',
 'B06X3ZLLXC good choice',
 'B06XX2CQRS good choice',
 'B06XT63P1X good choice',
 'B01M31PAOU good choice',
 'B074B65ZMT good choice',
 'B00DGF7JHS good choice',
 'B074P8BJSG good choice',
 'B01MDRB8AB good choice',
 'B003BSSERO good choice',
 'B074XGL1B8 good choice',
 'B01EKZZ0J6 good choice',
 'B07568NZX4 good choice',
 'B06ZZQ2CWS good choice',
 'B0725PVXS8 good choice',
 'B01FM4C1GE good choice',
 'B071VGTT2J good choice',
 'B01M1HN897 long sleeve suede top in plus sizefashion comes in four sizes and these tops with leather are loose fit very comfortable laser cut design end of the sleeves and front and back bottom paisley cut design front is v neck suede suede with lace up trim which completes this top very rare and only few pieces are left 

In [ ]:
import pickle
with open('review_emb_map.pickle', 'wb') as f:
        pickle.dump(review_emb_map, f)

In [ ]:
count = 0
for ele in concat_list:
  if ele == 'good choice':
    pass
  else:
    count += 1
    print(ele)
# print(count)

this personalized special olympics world games 2015 tshirt is great fun and a perfect gift for a loverfriendrelativesdressed on your own personalized tshirt is really cool too
b more information bbr feianna is an garment company  which is focus on the clothing design and production br we paid much attentions to the quality of our products and we will check every package br before shipping it outbr wish you have a happy shopping experience here br br choose a cotton linen cloth is choosing a quality and comfortable life lets close to natural br the charm isirresistiblebr usually feiannas cotton and linen clothing is casualmaxiplus sizefeatured br it can suitable for any kind of body typebr please compare detail size with yours it is importantbr 13cm differs is unavoidable due to different measure waythanks for you understandingbr wed like to help to choose a suitable size for you just leave message about your weight and heightbr
comfortable shortsleeved tshirt with a vneck in organic co